In [ ]:
import glob
import pandas as pd
import uproot
import uproot_methods
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable


plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

import numpy as np
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import interpolate

import itertools

import functions_giammi as fugi
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

dirN2 = r"../PYTHON_graphs/OUTPUTS/N2_calibration/"
allfiles = (glob.glob("../PYTHON_graphs/DATA/Standard_grpahs/*.root"))

cmap_viridis = mpl.cm.get_cmap('viridis')

In [ ]:
allfiles

In [ ]:
elec_lin = uproot.open(allfiles[0])
pipico_n2 = uproot.open(allfiles[1])
recoil_sum_w = uproot.open(allfiles[2])
recoil_xy = uproot.open(allfiles[3])
pipico_n2_momcon = uproot.open(allfiles[4])
elec_sum_w = uproot.open(allfiles[5])
elec_xy = uproot.open(allfiles[6])

In [ ]:
#N2
w_layer,tsum,counts_el_sum=fugi.import_TH2Dgeneric(elec_sum_w,elec_sum_w.keys()[0])
x_el,y_el,counts_el_xy=fugi.import_TH2Dgeneric(elec_xy,elec_xy.keys()[0])
x_rec,y_rec,counts_rec_xy=fugi.import_TH2Dgeneric(recoil_xy,recoil_xy.keys()[0])
x_el_lin,y_el_lin,counts_el_lin=fugi.import_TH2Dgeneric(elec_lin,elec_lin.keys()[0])
tof1,tof2,counts_pipico=fugi.import_TH2Dgeneric(pipico_n2_momcon,pipico_n2_momcon.keys()[0])

In [ ]:
#3D MFPAD
fig,ax= plt.subplots(1, figsize=(12, 10))

cs=ax.contourf(w_layer,tsum,np.ma.masked_array(counts_el_sum.T,counts_el_sum.T<3), levels=40)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('w-layer [ns]')
ax.set_ylabel('time sum [ns]')
ax.set_ylim(-1,1)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticklabels(["min","max"])
cbar.set_ticks([counts_el_sum.min(),counts_el_sum.max()])
cbar.ax.set_ylabel('normalized counts')

# fig.savefig(dir+"MFPAD3d.png", dpi=300, transparent=False)
# fig.savefig(dir+"MFPAD3d.svg", dpi=300, transparent=False)

In [ ]:
xy_el=np.array(list(itertools.product(x_el, y_el)))
X_el=np.array([col[0] for col in xy_el])
Y_el=np.array([col[1] for col in xy_el])

xy_rec=np.array(list(itertools.product(x_rec, y_rec)))
X_rec=np.array([col[0] for col in xy_rec])
Y_rec=np.array([col[1] for col in xy_rec])

In [ ]:
#3D MFPAD
counts_el_xy_new=np.ma.masked_where(np.sqrt(X_el**2+Y_el**2)<25,counts_el_xy.reshape(-1)).reshape(len(x_el),len(y_el))

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())
cs=ax.contourf(x_el,y_el,np.ma.masked_array(counts_el_xy_new, counts_el_xy_new<=5), levels=20)
# ax.set_title("fish fillet y-axis")


ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)
ax.plot([0, -19], [0, -33], color='black', linewidth=3., linestyle='--', alpha=0.7)
ax.plot([0, -33], [0, 19.5], color='black', linewidth=3., linestyle='--', alpha=0.7)
ax.plot([0, 33], [0, 20], color='black', linewidth=3., linestyle='--', alpha=0.7)
ax.grid()


cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticklabels(["min","max"])
# cbar.set_ticks([counts_el_xy.min(),counts_el_xy.max()])
# cbar.set_ticks([0,counts_el_xy.max()])

cbar.ax.set_ylabel('normalized counts')

# fig.savefig(dir+"MFPAD3d.png", dpi=300, transparent=False)
# fig.savefig(dir+"MFPAD3d.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
import matplotlib.colors as mcolors
counts_rec_xy_new=np.ma.masked_where(np.sqrt(X_rec**2+Y_rec**2)<20,counts_rec_xy.reshape(-1)).reshape(len(x_rec),len(y_rec))

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_rec.max()/y_rec.max())
# cs=ax.contourf(x_rec,y_rec,np.ma.masked_array(counts_rec_xy_new, counts_rec_xy_new<=5), levels=20)
cs=ax.contourf(x_rec,y_rec,np.ma.masked_array(counts_rec_xy_new, counts_rec_xy_new<=5), locator=mpl.ticker.LogLocator())
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)
# ax.plot([0, -19], [0, -33], color='black', linewidth=3., linestyle='--', alpha=0.7)
# ax.plot([0, -33], [0, 19.5], color='black', linewidth=3., linestyle='--', alpha=0.7)
# ax.plot([0, 33], [0, 20], color='black', linewidth=3., linestyle='--', alpha=0.7)
ax.grid()

norm = mcolors.LogNorm(np.ma.masked_array(counts_rec_xy_new, counts_rec_xy_new<=5).min(), np.ma.masked_array(counts_rec_xy_new, counts_rec_xy_new<=5).max())

im = ax.pcolormesh(np.ma.masked_array(counts_rec_xy_new, counts_rec_xy_new<=5), norm=norm)
# im.set_norm(norm)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticklabels(["min","max"])
# cbar.set_ticks([counts_el_xy.min(),counts_el_xy.max()])
# cbar.set_ticks([0,counts_el_xy.max()])

cbar.ax.set_ylabel('normalized counts')

# fig.savefig(dir+"MFPAD3d.png", dpi=300, transparent=False)
# fig.savefig(dir+"MFPAD3d.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())
cs=ax.contourf(x_el,y_el,np.ma.masked_array(counts_el_xy.T,counts_el_xy.T<1), locator=mpl.ticker.LogLocator())
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticklabels(["min","max"])
# cbar.set_ticks([counts_el_xy.min(),counts_el_xy.max()])
cbar.set_ticks([1,counts_el_xy.max()])

cbar.ax.set_ylabel('normalized counts')

# fig.savefig(dir+"MFPAD3d.png", dpi=300, transparent=False)
# fig.savefig(dir+"MFPAD3d.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())
cs=ax.contourf(x_el,y_el,np.ma.masked_array(counts_el_xy.T,counts_el_xy.T<1), levels=20)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticklabels(["min","max"])
cbar.set_ticks([counts_el_xy.min(),counts_el_xy.max()])
# cbar.set_ticks([1,counts_el_xy.max()])

cbar.ax.set_ylabel('normalized counts')

# fig.savefig(dir+"MFPAD3d.png", dpi=300, transparent=False)
# fig.savefig(dir+"MFPAD3d.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
x_el_lin_r=fugi.remap(x_el_lin,-45,45)
y_el_lin_r=fugi.remap(y_el_lin,-45,45)

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())
cs=ax.contourf(x_el_lin_r,y_el_lin_r,np.ma.masked_array(counts_el_lin.T,counts_el_lin.T<0.00001), levels=20)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
# cbar.set_ticklabels(["min","max"])
# cbar.set_ticks([counts_el_lin.min(),counts_el_lin.max()])
# cbar.set_ticks([1,counts_el_xy.max()])

cbar.ax.set_ylabel('deviation from linearity')

# fig.savefig(dir+"MFPAD3d.png", dpi=300, transparent=False)
# fig.savefig(dir+"MFPAD3d.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(tof1.max()/tof2.max())
cs=ax.contourf(tof1,tof2,np.ma.masked_array(counts_pipico.T,counts_pipico.T<1), levels=20)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('tof1 [ns]')
ax.set_ylabel('tof2 [ns]')
ax.set_aspect(aspect)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticklabels(["min","max"])
cbar.set_ticks([counts_pipico.min(),counts_pipico.max()])
# cbar.set_ticks([1,counts_el_xy.max()])

cbar.ax.set_ylabel('normalized counts')

# fig.savefig(dir+"MFPAD3d.png", dpi=300, transparent=False)
# fig.savefig(dir+"MFPAD3d.svg", dpi=300, transparent=False)